<a href="https://colab.research.google.com/github/zhestyatsky/bachelor-degree-research/blob/main/src/xlm_roberta_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile requirements.txt
pytorch-lightning
scikit-learn
transformers
sentencepiece

Writing requirements.txt


In [ ]:
!pip install --upgrade -r requirements.txt

     |████████████████████████████████| 675kB 8.9MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 1.5MB 56.0MB/s 
     |████████████████████████████████| 1.1MB 46.7MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 829kB 48.2MB/s 
     |████████████████████████████████| 276kB 52.1MB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=518ac597c81127f1c793186cb704cfa1af8627e6f105d10858ea5ff81f5930f4
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=662cce8bb902da245ed75f30cd3e91ca83a68326f920505f205471b0a635c2ab
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfe

In [ ]:
!mkdir data && git clone https://github.com/SapienzaNLP/mcl-wic.git data && cd data && unzip '*.zip' && rm *.zip && ls

Cloning into 'data'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 118 (delta 9), reused 0 (delta 0), pack-reused 96
Receiving objects: 100% (118/118), 3.62 MiB | 35.63 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Archive:  SemEval-2021_MCL-WiC_training.zip
   creating: training/
   creating: training/multilingual/
  inflating: training/multilingual/training.en-en.data  
  inflating: training/multilingual/training.en-en.gold  
  inflating: training/LICENSE.txt    

Archive:  SemEval-2021_MCL-WiC_trial.zip
   creating: trial/
   creating: trial/crosslingual/
  inflating: trial/crosslingual/trial.en-fr.data  
  inflating: trial/crosslingual/trial.en-ar.gold  
  inflating: trial/crosslingual/trial.en-zh.gold  
  inflating: trial/crosslingual/trial.en-fr.gold  
  inflating: trial/crosslingual/trial.en-ru.gold  
  inflating: trial/crosslingual/trial.en-ar.data  
  inflating: trial/c

In [ ]:
import json
import pandas as pd

def read_train_data():
    with open('data/training/multilingual/training.en-en.data') as f:
        df_train = pd.DataFrame(json.load(f))
    with open('data/training/multilingual/training.en-en.gold') as f:
        df_train = df_train.merge(pd.DataFrame(json.load(f)))

    df_train["tag"] = df_train["tag"].replace({"T": 1, "F": 0})
    
    return df_train


df_train = read_train_data()
df_train.head()

,id,lemma,pos,sentence1,sentence2,start1,end1,start2,end2,tag
0,training.en-en.0,play,NOUN,In that context of coordination and integratio...,A musical play on the same subject was also st...,69,73,10,14,0
1,training.en-en.1,play,NOUN,In that context of coordination and integratio...,"In schools, when water is needed, it is girls ...",69,73,112,116,0
2,training.en-en.2,esteem,NOUN,We would also like to convey our esteem and co...,"Father Lini said that, because of that, the Un...",33,39,106,112,1
3,training.en-en.3,esteem,NOUN,We would also like to convey our esteem and co...,This attests to the esteem and trust enjoyed b...,33,39,20,26,1
4,training.en-en.4,holder,NOUN,This growth is the direct result of the increa...,A person may be either the holder of an option...,74,81,27,33,1


In [ ]:
import glob


def read_dev_data():
    df_dev = pd.DataFrame()
    for file in glob.glob('data/development/multilingual/*.data'):
        with open(file, 'r') as f:
          df_dev = df_dev.append(pd.DataFrame(json.load(f)))

    df_dev_tags = pd.DataFrame()
    for file in glob.glob('data/development/multilingual/*.gold'):
        with open(file, 'r') as f:
            df_dev_tags = df_dev_tags.append(pd.DataFrame(json.load(f)))

    df_dev = df_dev.merge(df_dev_tags)
    df_dev["tag"] = df_dev["tag"].replace({"T": 1, "F": 0})
    
    return df_dev

df_dev = read_dev_data()

In [ ]:
from sklearn.model_selection import train_test_split

df_val, df_test = train_test_split(df_dev, test_size=0.5, random_state=1)
df_val.shape, df_test.shape

((2500, 10), (2500, 10))

In [ ]:
train_labels = df_train["tag"].tolist()
val_labels = df_val["tag"].tolist()
test_labels = df_test["tag"].tolist()

train_texts = [(s1, s2) for s1, s2 in zip(df_train["sentence1"], df_train["sentence2"])]
val_texts = [(s1, s2) for s1, s2 in zip(df_val["sentence1"], df_val["sentence2"])]
test_texts = [(s1, s2) for s1, s2 in zip(df_test["sentence1"], df_test["sentence2"])]

train_texts[0], train_labels[0]

(('In that context of coordination and integration, Bolivia holds a key play in any process of infrastructure development.',
  'A musical play on the same subject was also staged in Kathmandu for three days.'),
 0)

In [ ]:
import json
from torch.utils.data import Dataset, RandomSampler, DataLoader
from transformers import XLMRobertaTokenizer

class XLMRDataset(Dataset):
    def __init__(self, texts, labels, max_tokens=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")
        self.max_tokens = max_tokens
    
    def _tokenize(self, sentence):
        return self.tokenizer(sentence,
                              add_special_tokens=True,
                              max_length=self.max_tokens,
                              padding="max_length",
                              truncation=True)

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        first_sentence, second_sentence = self.texts[index]
        first_input = self._tokenize(first_sentence)
        second_input = self._tokenize(second_sentence)
        
        inputs = (torch.tensor(first_input["input_ids"]), torch.tensor(second_input["input_ids"]))
        attention_masks = (torch.tensor(first_input["attention_mask"]), torch.tensor(second_input["attention_mask"]))

        return inputs, attention_masks, torch.tensor(self.labels[index], dtype=torch.float)

MAX_TOKENS = 64
BATCH_SIZE = 16
EPOCHS = 4

xlm_train_data = XLMRDataset(train_texts, train_labels, MAX_TOKENS)
xlm_train_sampler = RandomSampler(xlm_train_data)
xlm_train_loader = DataLoader(xlm_train_data, batch_size=BATCH_SIZE, sampler=xlm_train_sampler)

xlm_val_data = XLMRDataset(val_texts, val_labels, MAX_TOKENS)
xlm_val_loader = DataLoader(xlm_val_data, batch_size=BATCH_SIZE)

xlm_test_data = XLMRDataset(test_texts, test_labels, MAX_TOKENS)
xlm_test_loader = DataLoader(xlm_test_data, batch_size=BATCH_SIZE)

TOTAL_STEPS = (len(xlm_train_data) // BATCH_SIZE + 1) * EPOCHS

In [ ]:
import torch
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping  
from transformers import XLMRobertaModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score 
from torch import nn

class XLMRClassifier(LightningModule):
    def __init__(self, model_path="xlm-roberta-large"):
        super(XLMRClassifier, self).__init__()

        self.model = XLMRobertaModel.from_pretrained(model_path)
        self.linear = nn.Linear(2 * self.model.get_input_embeddings().embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCELoss()

        self.save_hyperparameters() 
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()

    def forward(self, input_ids, attention_mask):
        first_sentence_embeddings = self.model(input_ids[0], attention_mask[0]).pooler_output
        second_sentence_embeddings = self.model(input_ids[1], attention_mask[1]).pooler_output
        embeddings = torch.cat((first_sentence_embeddings, second_sentence_embeddings), 1)
        outputs = self.sigmoid(self.linear(embeddings)).view(-1)
        return outputs

    def training_step(self, batch, _):
        inputs, attn, labels = batch
        outputs = self(inputs, attn)
        return self.loss(outputs, labels)
    
    def validation_step(self, batch, _):
        inputs, attn, labels = batch
        outputs = self(inputs, attn)
        logits = (outputs>0.5).float()

        self.valid_accuracy.update(logits, labels)
        self.log("val_acc", self.valid_accuracy)

        loss = self.loss(outputs, labels)
        self.log("val_loss", loss, prog_bar=True)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, attn, labels = batch
        outputs = self(inputs, attn)
        logits = (outputs>0.5).float()

        self.test_accuracy.update(logits, labels)
        self.log("test_acc", self.test_accuracy)

        loss = self.loss(outputs, labels)
        self.log("test_loss", loss, prog_bar=True)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-3)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=TOTAL_STEPS)
        return [optimizer], [scheduler]


clf = XLMRClassifier()
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=6,
    max_epochs=EPOCHS,
    callbacks=[early_stop_callback],
    val_check_interval=0.25)
trainer.fit(clf, xlm_train_loader, xlm_val_loader)
trainer.test(clf, xlm_test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type            | Params
---------------------------------------------------
0 | model          | XLMRobertaModel | 559 M 
1 | linear         | Linear          | 2.0 K 
2 | sigmoid        | Sigmoid         | 0     
3 | loss           | BCELoss         | 0     
4 | valid_accuracy | Accuracy        | 0     
5 | test_accuracy  | Accuracy        | 0     
---------------------------------------------------
559 M     Trainable params
0         Non-trainable params
559 M     Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4924, device='cuda:0'),
 'test_acc_epoch': tensor(0.4924, device='cuda:0'),
 'test_loss': tensor(0.6982, device='cuda:0'),
 'val_acc': tensor(0.5076, device='cuda:0'),
 'val_acc_epoch': tensor(0.5076, device='cuda:0'),
 'val_loss': tensor(0.6956, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.49239999055862427,
  'test_acc_epoch': 0.49239999055862427,
  'test_loss': 0.6981812715530396,
  'val_acc': 0.5076000094413757,
  'val_acc_epoch': 0.5076000094413757,
  'val_loss': 0.6955567598342896}]